In [1]:
# 正则表达式规则构建
res = {
r"^(Your *)*Roles* (and|&) Responsibilit(y|ies) *.*[:?]*\n",
r'^Job Descriptions* *[:?]*\n',
r'^(Your|Job|Main|Key)* *Responsibilit(y|ies) *.*[:?]*\n',
r'^(Your|Job|Main|Key)* *Accountabilit(y|ies) *.*[:?]*\n'
r'^What you( will|\'ll) (do|be doing).*\n',
r'^In this (new )*(position|role),* you( will|\'ll)( get to)* *(\.\.\.|:|\?)*\n',
r'^Get to know the role *[:?]*\n',
r'^(About *)*the role *[:?]*\n',
r'^Your role at .*:*\n',
r"^Your role *.*:*\n",
r"^The job *:*\n",
r"^(Your )*(Job )*scope *:*\n",
r"What will you do *[:?]* *\n",
r"^Dut(y|ies) (And|&) Responsibilit(y|ies) *:*\n"
}

edu = {
r"^(Language|Basic|Essential|Key)* *(Job)* *Requirements*.*[:?]*\n",
r"^Qualifications*( for this role)* *[:?]*\n",
r"^The must(-| )haves* *[:?]*\n",
r"^Who you( are|\'re) *[:?]*\n",
r"^About you *[:?]*\n",
r"^(Your|Candidate) (background|profile) *:*\n",
r"^You bring most of this *:*\n",
r"^What you will (need to succeed|bring) *[:?]*\n",
r"^What we expect from you *[:?]*\n",
r"^Who are we looking for *[:?]*\n",
r"^To be a successful candidate *:*\n",
r"^Qualifications* */ *Requirements* *:*\n",
r"^Skills*,* Knowledge (And|&) Expertise *:*\n",
r"^What will you need *[:?]* *\n",
r"^What you( will|\'ll) need *[:?]* *\n",
r"^(Basic )*Skills* (and|&) Qualifications* :*\n"
}

skill = {
r"^(Required )*Skills* *:* *\n",
r"^(Personal|Your)* *Attributes* *:* *\n"
}


us = {
r"^Who we are *[:?]*\n",
r"^About us *[:?]*\n",
r"^About (the )*\{*company\}* *[:?]*\n",
r"^Company Descriptions* *:*\n",
r"^What we do *[:?]*\n",
r"^Our company *:*\n",
r"^Company Overview *.*:*\n",
}

loc = {
r"(?<=Work Location:).*\n",
r"(?<=Working Location:).*\n",
r'(?<=Location:).*?(?=[\.\n])',
r"(?<=based in )\w",
'Bayan Lepas, Penang',
'Penang, Malaysia',
'Penang',
'Johor Bahru, Malaysia',
'Johor Bahru',
}

other = {
r"^Additional Information *:*\n"
}

In [4]:
import numpy as np
import pandas as pd
import re
from IPython.display import clear_output

# 选择清洗的数据集
def select_file():
    value = input("Select file:")
    pathway = input("Select clean or raw:")
    if pathway == "clean":
        path = f'D:\其桦D盘\其桦大学\毕业论文\scrapper\clean_result\\res_{value}.csv'
    else:
        path = f'D:\其桦D盘\其桦大学\毕业论文\scrapper\scrap_result\\res_{value}.csv'
    df = pd.read_csv(path, index_col=0, sep="|", dtype=str)
    print(f'file read!')
    return df, value

# 选择行
def choose_row():
    row_number = input("Select row number:")
    return int(row_number)

# 替换标识符和文本清洗
def replace_token(df):
    if df is None:
        return
    df.dropna(inplace=True, how='all')
    df = df.drop_duplicates()
    web_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    hashtags = r"(#\w+)"
    email_regex = r"[a-zA-Z0-9.!#$%&’*+/=?^_`{|}~-]+@[a-zA-Z0-9-]+(?:\.[a-zA-Z0-9-]+)*"
    special_patt = r'^\[Apply now at {website}\]\n'
    try:
        for index, row in df.iterrows():
            
            row["Job desc"] = row["Job desc"].replace(str(row["Job title"]), "{job}")
            row["Job desc"] = row["Job desc"].replace(str(row["Company name"]), "{company}")
            row["Job desc"] = row["Job desc"].replace(str(row["Location"]), "{location}")
            row["Job desc"] = re.sub(web_regex, "{website}", row["Job desc"])
            row["Job desc"] = re.sub("Jobstore\.com", "{website}", row["Job desc"])
            row["Job desc"] = re.sub(hashtags, "", row["Job desc"])
            row["Job desc"] = re.sub(email_regex, "{email}", row["Job desc"])
            for i in res:
                row["Job desc"] = re.sub(i, "Responsibilities:\n", row["Job desc"], flags=re.M|re.I)
            for i in edu:
                row["Job desc"] = re.sub(i, "Education, Qualifications & Experience:\n", row["Job desc"], flags=re.M|re.I)
            for i in skill:
                row["Job desc"] = re.sub(i, "Skills & Abilities:\n", row["Job desc"], flags=re.M|re.I)        
            for i in loc:
                row["Job desc"] = re.sub(i, "{location}", row["Job desc"], flags=re.M|re.I)
            for i in us:
                row["Job desc"] = re.sub(i, "About us:\n", row["Job desc"], flags=re.M|re.I)
            for i in other:
                row["Job desc"] = re.sub(i, "Other:\n", row["Job desc"], flags=re.M|re.I)
            if re.match(special_patt, row["Job desc"], flags=re.M|re.I):
                l, r = re.split(special_patt, row["Job desc"], maxsplit=1, flags=re.M|re.I)
                row["Job desc"] = "Responsibilities:\n" + l + "[Apply now at {website}]\n" + "Education, Qualifications & Experience:\n" + r
            row["Job desc"] = re.sub(r"This job is sourced from a job board. Learn more\n", "", row["Job desc"])
            row["Job desc"] = re.sub(r"^What('s| is) on offer\?\n", "Other:\nWhat's on Offer?\n", row["Job desc"])
            row["Job desc"] = re.sub(r"^(Perks*)* *(and|&)* *(Job )*Ben[ei]fits*:*\n", "Other:\nJob Benefits:\n", row["Job desc"])
            row["Job desc"] = re.sub(r"^What's in it for you\?*:*\n", "Other:\nWhat's in it for you:\n", row["Job desc"])
            row["Job desc"] = re.sub("::", ":", row["Job desc"])
            row["Job desc"] = re.sub(r"\?€\?", "'", row["Job desc"])
            row["Job desc"] = re.sub("Responsibilities:\n+Responsibilities:\n", "Responsibilities:\n", row["Job desc"])
            row["Job desc"] = re.sub("Education, Qualifications & Experience:\n+Education, Qualifications & Experience:\n", "Education, Qualifications & Experience:\n", row["Job desc"])
    except Exception as e:
        raise(e)
    return df

# 替换文字
def replace_word(df):
    patt = input("Find:")
    text = input("Replace with:")
    for index, row in df.iterrows():
        row["Job desc"] = re.sub(patt, text, row["Job desc"], flags=re.M|re.I|re.S)
    return df

# 选择特定企业进行清洗
def choose_company(df):
    new_df = df.copy()
    company = input('Choose company:')
    new_df = df.loc[df['Company name']== company]
    new_df = looping_app(new_df, comp_mode=True)
    confirm = input('Confirm changes?')
    if confirm == 'y':
        df.loc[df['Company name']==company, 'Job desc'] = new_df.loc[:,'Job desc']
    return df

# 加入文本
def add_comp_intro(df):
    df.loc[:,'Job desc'] = 'About us:\n' + df['Job desc']
    return df
    
# 构建循环
def loop_dataframe(df, row):
    new_df = df.copy()
    while row < len(new_df):
        clear_output()
        print(df.iloc[row, 8])
        new_text = input("New text:\n")
        if new_text == "q":
            break
        elif new_text == "b":
            row -= 1
            new_df.iloc[row, 8] = df.iloc[row, 8]
        elif new_text == "c":
            row = choose_row()
        elif new_text == "n":
            row += 1
        else:
            new_df.iloc[row, 8] = new_text
            row += 1
    if row >= len(new_df):
        print("Current row exceeded dataframe length.")
        row = 0
    return new_df, df, row

# 存储数据集
def save_df(df, value):
    path = f'D:\其桦D盘\其桦大学\毕业论文\scrapper\clean_result\\res_{value}.csv'
    df.to_csv(path, sep="|")

# 函数封装
def looping_app(df=None, comp_mode=False):
    row = 0
    while True:
        try:
            command = input("Command:")
            if command == "r":
                df = replace_token(df)
            elif command == "f":
                df = replace_word(df)
            elif command == "l":
                df, old_df, row = loop_dataframe(df, row)
            elif command == "p":
                print(df.iloc[row,:])
            elif command == "a":
                df = add_comp_intro(df)
            elif command == "q":
                break
            elif comp_mode:
                continue
            elif command == "s":
                save_df(df, value)
            elif command == "o":
                df, value = select_file()
                row = 0
            elif command == "comp":
                df = choose_company(df)
            else:
                try:
                    eval(command)
                except:
                    pass
        except Exception as e:
            print(e)
    return df

In [7]:
looping_app()
print("done")

Salary package inclusive of basic + allowance + commission

About us:
Ultima Markets is an international online trading platform, offering over 250 financial instruments such as forex, metals & commodities, indices, shares, CFDs, cryptos etc. We are incorporated in Australia and are regulated by Australian Securities and Investments Commissions (ASIC). We are one of the subsidiaries of an Australian MNC with presence in many countries. Our business is expanding rapidly and we aim to set up a central hub in Kuala Lumpur to work with our team in APAC region such as Hong Kong, China and South East Asia countries. We are actively seeking out for talent to join our pioneer team in Malaysia.

You can visit our website: {website} to understand more about our company.

Responsibilities:
Proactively source and secure potential institutional clients throughout APAC market
Manage customer relationship, retaining and expanding the existing relationship and drive volume and revenue continuously
Col